<a href="https://colab.research.google.com/github/dtoralg/INESDI_Data-Science_ML_IA/blob/main/%5B04%5D%20-%20Modelos%20No%20Supervisados/No_supervisados_Ejercicio_8_kmeans_wholesale_customers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# No supervisados - Ejercicio 8: kmeans_wholesale_customers.ipynb

Este notebook es **We do**: lo hacemos juntos. Algunas celdas están resueltas para guiarte en las partes complejas, pero encontrarás comentarios `# TODO` dentro del código que debes completar durante la sesión. Ejecuta las celdas en orden, experimenta con los parámetros y anota tus decisiones.

## Objetivos

- Cargar y explorar el dataset Wholesale Customers.
- Preparar los datos (selección de features y escalado).
- Elegir un `k` razonable con Elbow + Silhouette y entrenar KMeans.
- Comparar KMeans con DBSCAN y detectar outliers con LOF.
- Perfilar clusters y redactar acciones de negocio basadas en los segmentos.

In [ ]:
# Librerías y configuración (resuelto)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors, LocalOutlierFactor
from sklearn.metrics import silhouette_score, adjusted_rand_score

np.random.seed(42)
print('Librerías importadas correctamente')

### Carga de datos

Carga el dataset desde una URL pública o desde tu copia local. Si trabajas en el aula, puedes pre-subir el CSV al entorno.

In [ ]:
url = 'https://raw.githubusercontent.com/dtoralg/INESDI_Data-Science_ML_IA/refs/heads/main/%5B04%5D%20-%20Modelos%20No%20Supervisados/Wholesale%20customers%20data.csv'
df = pd.read_csv(url)
display(df.head())
print(df.shape)

### Exploración inicial

Revisa tipos, nulos, estadísticas y correlaciones para entender las variables de gasto.

In [ ]:
# TODO:
# 1) Muestra df.info() y df.describe().T
# 2) Calcula el % de nulos por columna: (df.isnull().mean()*100).round(2)
# 3) Dibuja un heatmap de correlación entre las variables de gasto
# 4) Opcional: pairplot sobre un subconjunto para ver relaciones bivariadas

# Resultado esperado: identificar las columnas numéricas de gasto y si necesitas imputar.

### Selección de features

Identifica las columnas numéricas que representan gasto. En general usaremos las columnas: `Fresh`, `Milk`, `Grocery`, `Frozen`, `Detergents_Paper`, `Delicassen`.

In [ ]:
# TODO:
# - Define `features` como la lista de columnas de gasto presentes en tu df.
# - Crea X = df[features].copy()
# - Comprueba X.describe().T

# Ejemplo guía:
# features = ['Fresh','Milk','Grocery','Frozen','Detergents_Paper','Delicassen']
# X = df[features].copy()
# display(X.head())

### Escalado

Escalamos con StandardScaler. En esta celda he creado el scaler; tú debes aplicarlo y guardar `X_scaled` como DataFrame con los mismos nombres de columnas.

In [ ]:
# Resuelto: creamos el scaler
scaler = StandardScaler()

# TODO: aplica scaler sobre X y guarda X_scaled como DataFrame con columnas=features
# Ejemplo guía:
# X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=features)
# display(X_scaled.describe().T)

# Comprueba que X_scaled tiene la misma cantidad de filas que X

### Determinar k: Elbow + Silhouette

Probamos varios k y observamos inertia y silhouette. No buscamos automatizar: interpretamos los gráficos y decidimos juntos.

In [ ]:
# TODO:
# - Recorre k en range(2,11)
# - Para cada k entrena KMeans(n_clusters=k, random_state=42, n_init=10) sobre X_scaled
# - Guarda inertia y silhouette_score (silhouette_score requiere al menos 2 clusters)
# - Construye un DataFrame con columnas ['k','inertia','silhouette'] y plotea ambos gráficos

# Pista: utiliza try/except para silhouette_score por si falla en algún k

### Entrenar KMeans y perfilar clusters

Entrena KMeans con el k que decidamos en clase y calcula el perfil medio por cluster para interpretar segmentos.

In [ ]:
# TODO:
# - Define k_chosen (elige basado en la celda anterior)
# - Entrena kmeans = KMeans(n_clusters=k_chosen, random_state=42, n_init=10) y obtén labels_km = kmeans.fit_predict(X_scaled)
# - Añade df['cluster_km'] = labels_km
# - Muestra tamaño de clusters y df.groupby('cluster_km')[features].mean().round(2)

# Resuelto (ejemplo de cómo perfilar):
# display(df.groupby('cluster_km')[features].mean().round(2))

# Nota: completa los pasos para que lo puedas ejecutar en tu entorno

### PCA para visualización

Usa PCA para visualizar en 2D. En clase proyectaremos y compararemos visualmente los clusters.

In [ ]:
# Resuelto: creamos un PCA y proyectamos (ejecuta tal cual si X_scaled está definido)
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
# TODO: si ya tienes X_scaled (DataFrame o array), usa pca.fit_transform(X_scaled) y guarda en X_pca2
# Ejemplo (si X_scaled es DataFrame):
# X_pca2 = pca.fit_transform(X_scaled)
# plt.figure(figsize=(8,6)); sns.scatterplot(x=X_pca2[:,0], y=X_pca2[:,1], hue=df['cluster_km'].astype(str), palette='tab10'); plt.show()


### Comparación con DBSCAN

Hacemos un k-distance plot para estimar eps y probamos DBSCAN con un par de configuraciones. Luego medimos similitud con KMeans.

In [ ]:
# TODO:
# - Calcula k-distance plot con NearestNeighbors(n_neighbors=5)
# - Estima un rango de eps y prueba DBSCAN(eps=..., min_samples=5)
# - Guarda labels_db y añade df['cluster_db'] = labels_db
# - Muestra np.unique(labels_db) y pd.Series(labels_db).value_counts()
# - Si tienes labels_km y labels_db, calcula adjusted_rand_score(labels_km, labels_db)

# Resuelto: ejemplo de creación de NearestNeighbors
# neigh = NearestNeighbors(n_neighbors=5)
# distances, indices = neigh.fit(X_scaled).kneighbors(X_scaled)
# distances = np.sort(distances[:, -1])
# plt.plot(distances); plt.title('k-distance plot'); plt.show()


### LOF: detección de outliers

Aplicaremos Local Outlier Factor para detectar outliers sobre las mismas features. Observaremos si los outliers coinciden con el ruido detectado por DBSCAN.

In [ ]:
# TODO:
# - Ajusta lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
# - Obtén lof_labels = lof.fit_predict(X_scaled) y añade df['lof_label'] = lof_labels
# - Muestra pd.Series(lof_labels).value_counts() y algunos ejemplos df[df['lof_label']==-1].head()
# - Visualiza en PCA 2D coloreando por lof_label

# Resuelto: ejemplo de cómo instanciar LOF (no ejecuta fit_predict aquí)
# lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)


### Perfilado y acciones

Perfila los clusters y revisa los outliers. A partir de esta información, redacta una recomendación de negocio por cluster y qué hacer con los outliers.

In [ ]:
# TODO:
# - Muestra medias por cluster (df.groupby('cluster_km')[features].mean())
# - Lista ejemplos de outliers (df[df['lof_label']==-1])
# - Escribe un breve párrafo con 2 acciones concretas por cluster y 2 acciones para los outliers

# Resuelto: template para mostrar medias
# display(df.groupby('cluster_km')[features].mean().round(2))

### Entrega

En la última celda escribe un resumen (4-6 líneas) con: k elegido y por qué, parámetros DBSCAN y LOF probados, principales hallazgos y 2 recomendaciones de negocio. Guarda y descarga el notebook.

In [ ]:
# Escribe tu resumen aquí (sustituye el texto entre las comillas):
resumen = '''

'''
print(resumen)